# WEB SCRAPING FROM ROTTENTOMATOES MOVIE WEBSITE

# CREDITS
This work is inspired by a great source done before:
- https://www.udemy.com/course/web-scraping-and-api-fundamentals-in-python/

# INTRODUCTION
Are you tired of manually copying and pasting values in a spreadsheet? Do you want to learn how to obtain interesting, real-time and even rare information from the internet with a simple script?

Web Scraping is a technique for obtaining information from web pages or other sources of data, such as webpages, downloadable files, and APIs, through the use of intelligent automated programs. Web Scraping allows us to gather data from potentially hundreds or thousands of pages with a few lines of code. When it comes to data science – more and more data comes from external sources, thus knowing how to extract and structure that data quickly is an essential skill that will set you apart in the job market.

# LIBRARY

In [1]:
# Data Processing
import pandas as pd

# Data Scraping
import requests
from bs4 import BeautifulSoup

# Target URL

In [2]:
# Define the URL of the site
base_site = "https://editorial.rottentomatoes.com/guide/140-essential-action-movies-to-watch-now/2/"

In [3]:
# sending a request to the webpage
response = requests.get(base_site)
response.status_code

200

In [4]:
# get the HTML from the webpage
html = response.content

# CHOOSING A PARSER
- 3 Options: lxml (best), html5lib (second), html.parser (worst)

## A word of caution
- Beautiful Soup ranks the lxml parser as the best one.
- If a parser is not explicitly stated in the Beautiful Soup constructor,
- the best one available on the current machine is chosen.
- This means that the same piece of code can give different results on different computers.

## html.parser
-  When inspecting the file we see that HTML element is closed at the begining -- it parsed incorrectly!
-  Let's check another parser

In [5]:
# convert the HTML to a Beautiful Soup object
soup = BeautifulSoup(html, 'html.parser')

# Exporting the HTML to a file
with open('Rotten_tomatoes_page_2_HTML_Parser.html', 'wb') as file:
    file.write(soup.prettify('utf-8'))

## lxml
- By first accounts of inspecting the file everything seems fine

In [6]:
# convert the HTML to a BeatifulSoup object
soup = BeautifulSoup(html, 'lxml')

# Exporting the HTML to a file
with open('Rotten_tomatoes_page_2_LXML_Parser.html', 'wb') as file:
    file.write(soup.prettify('utf-8'))

# FINDING AN ELEMENT CONTAINING ALL THE DATA

In [7]:
# Find all div tags on the webpage containing the information we want to scrape
divs = soup.find_all("div", {"class": "col-sm-18 col-full-xs countdown-item-content"})
divs[:2]

[<div class="col-sm-18 col-full-xs countdown-item-content">
 <div class="row countdown-item-title-bar">
 <div class="col-sm-20 col-full-xs" style="height: 100%;">
 <div class="article_movie_title" style="float: left;">
 <div><h2><a href="https://www.rottentomatoes.com/m/1018009-running_scared/">Running Scared</a> <span class="subtle start-year">(1986)</span> <span class="icon tiny fresh" title="Fresh"></span> <span class="tMeterScore">60%</span></h2></div>
 </div>
 </div>
 <div class="col-sm-4 col-full-xs" style="height: 100%;">
 <div class="countdown-index">#140</div>
 </div>
 </div>
 <div class="row countdown-item-details">
 <div class="col-sm-24">
 <div class="info countdown-adjusted-score"><span class="descriptor">Adjusted Score: </span>61.167% <span class="glyphicon glyphicon-question-sign" data-html="true" data-original-title="The Adjusted Score comes from a weighted formula (Bayesian) that we use that accounts for variation in the number of reviews per movie." data-placement="to

# EXTRACTION OF ALL ELEMENTS FOR EACH MOVIE
- The title, year and score of each movie are contained in the 'h2' tags

In [8]:
# for instance, let's explore the first div
divs[0].find("h2")

<h2><a href="https://www.rottentomatoes.com/m/1018009-running_scared/">Running Scared</a> <span class="subtle start-year">(1986)</span> <span class="icon tiny fresh" title="Fresh"></span> <span class="tMeterScore">60%</span></h2>

In [9]:
# Extracting all 'h2' tags
headings = [div.find("h2") for div in divs]
headings[:5]

[<h2><a href="https://www.rottentomatoes.com/m/1018009-running_scared/">Running Scared</a> <span class="subtle start-year">(1986)</span> <span class="icon tiny fresh" title="Fresh"></span> <span class="tMeterScore">60%</span></h2>,
 <h2><a href="https://www.rottentomatoes.com/m/equilibrium/">Equilibrium</a> <span class="subtle start-year">(2002)</span> <span class="icon tiny rotten" title="Rotten"></span> <span class="tMeterScore">40%</span></h2>,
 <h2><a href="https://www.rottentomatoes.com/m/hero/">Hero</a> <span class="subtle start-year">(2004)</span> <span class="icon tiny certified" title="Certified Fresh"></span> <span class="tMeterScore">95%</span></h2>,
 <h2><a href="https://www.rottentomatoes.com/m/1017666-road_house/">Road House</a> <span class="subtle start-year">(1989)</span> <span class="icon tiny rotten" title="Rotten"></span> <span class="tMeterScore">39%</span></h2>,
 <h2><a href="https://www.rottentomatoes.com/m/unstoppable-2010/">Unstoppable</a> <span class="subtle st

In [10]:
# Inspecting the text inside the headings
[heading.text for heading in headings][:5]

['Running Scared (1986)  60%',
 'Equilibrium (2002)  40%',
 'Hero (2004)  95%',
 'Road House (1989)  39%',
 'Unstoppable (2010)  86%']

- It does contain the info we want to extract
- However, we need to obtain the title, year and score separately
- Let's inspect one heading to see if there is a way to distinguish between them

### Observation:
- The movie title is in the 'a' tag
- The year is in a 'span' with class 'start-year'
- The score is in a 'span' with class 'tMeterScore'

In [11]:
headings[0]

<h2><a href="https://www.rottentomatoes.com/m/1018009-running_scared/">Running Scared</a> <span class="subtle start-year">(1986)</span> <span class="icon tiny fresh" title="Fresh"></span> <span class="tMeterScore">60%</span></h2>

## 1. Title

In [12]:
# Let's check all heading links
[heading.find('a') for heading in headings][:5]

[<a href="https://www.rottentomatoes.com/m/1018009-running_scared/">Running Scared</a>,
 <a href="https://www.rottentomatoes.com/m/equilibrium/">Equilibrium</a>,
 <a href="https://www.rottentomatoes.com/m/hero/">Hero</a>,
 <a href="https://www.rottentomatoes.com/m/1017666-road_house/">Road House</a>,
 <a href="https://www.rottentomatoes.com/m/unstoppable-2010/">Unstoppable</a>]

### Filtering Title

In [13]:
# Obtaining the movie titles from the links
movie_names = [heading.find('a').string for heading in headings]
movie_names[:5]

['Running Scared', 'Equilibrium', 'Hero', 'Road House', 'Unstoppable']

## 2. Year

### Filtering Year

In [14]:
# Filtering only the spans containing the year
[heading.find("span", class_ = 'start-year') for heading in headings] [:5]

[<span class="subtle start-year">(1986)</span>,
 <span class="subtle start-year">(2002)</span>,
 <span class="subtle start-year">(2004)</span>,
 <span class="subtle start-year">(1989)</span>,
 <span class="subtle start-year">(2010)</span>]

In [15]:
# Extracting the year string
years = [heading.find("span", class_ = 'start-year').string for heading in headings]
years [:5]

['(1986)', '(2002)', '(2004)', '(1989)', '(2010)']

In [16]:
years[0]

'(1986)'

### Removing the Brackets from Year

#### Observation
- One way to remove the brackets is to drop the first and last symbol of the string
- However, this will break, if the format of the year is changed

In [17]:
years[0][1:-1]

'1986'

#### Observation
- Alternativelly, we can do it with the help of the strip() method (this is robust)
- It removes leading and trailing symbols from a string
- By default, it removes whitespace, but we can specify other symbols to strip

In [18]:
# Removing '('
print(years[0].strip('('))

# Removing ')'
print(years[0].strip(')'))

# Combining both
print(years[0].strip('()'))

1986)
(1986
1986


In [19]:
# Updating years with stripped values
years = [year.strip('()') for year in years]
years [:5]

['1986', '2002', '2004', '1989', '2010']

In [20]:
# Converting all the strings to integers
years = [int(year) for year in years]
years [:5]

[1986, 2002, 2004, 1989, 2010]

## 3. Score

### Filtering Score

In [21]:
# Filtering only the spans containing the score
[heading.find("span", class_ = 'tMeterScore') for heading in headings] [:5]

[<span class="tMeterScore">60%</span>,
 <span class="tMeterScore">40%</span>,
 <span class="tMeterScore">95%</span>,
 <span class="tMeterScore">39%</span>,
 <span class="tMeterScore">86%</span>]

In [22]:
# Extracting the score string
scores = [heading.find("span", class_ = 'tMeterScore').string for heading in headings]
scores [:5]

['60%', '40%', '95%', '39%', '86%']

### Removing the % Sign

In [23]:
# Removing the '%' sign
scores = [s.strip('%') for s in scores]
scores [:5]

['60', '40', '95', '39', '86']

In [24]:
# Converting each score to an integer
scores = [int(s) for s in scores]
scores [:5]

[60, 40, 95, 39, 86]

## 4. Critics Consensus
- inside a div with class 'critics-consensus'

In [25]:
# The critics consensus is located inside a 'div' tag with the class 'info critics-consensus'
# This can be found inside the original 'div's we scraped
divs [:1]

[<div class="col-sm-18 col-full-xs countdown-item-content">
 <div class="row countdown-item-title-bar">
 <div class="col-sm-20 col-full-xs" style="height: 100%;">
 <div class="article_movie_title" style="float: left;">
 <div><h2><a href="https://www.rottentomatoes.com/m/1018009-running_scared/">Running Scared</a> <span class="subtle start-year">(1986)</span> <span class="icon tiny fresh" title="Fresh"></span> <span class="tMeterScore">60%</span></h2></div>
 </div>
 </div>
 <div class="col-sm-4 col-full-xs" style="height: 100%;">
 <div class="countdown-index">#140</div>
 </div>
 </div>
 <div class="row countdown-item-details">
 <div class="col-sm-24">
 <div class="info countdown-adjusted-score"><span class="descriptor">Adjusted Score: </span>61.167% <span class="glyphicon glyphicon-question-sign" data-html="true" data-original-title="The Adjusted Score comes from a weighted formula (Bayesian) that we use that accounts for variation in the number of reviews per movie." data-placement="to

### Filtering Critics Consensus

In [26]:
# Getting the 'div' tags containing the critics consensus
consensus = [div.find("div", {"class": "info critics-consensus"}) for div in divs]
consensus [:5]

[<div class="info critics-consensus"><span class="descriptor">Critics Consensus:</span> Running Scared struggles to strike a consistent balance between violent action and humor, but the chemistry between its well-matched leads keeps things entertaining.</div>,
 <div class="info critics-consensus"><span class="descriptor">Critics Consensus:</span> Equilibrium is a reheated mishmash of other sci-fi movies.</div>,
 <div class="info critics-consensus"><span class="descriptor">Critics Consensus:</span> With death-defying action sequences and epic historic sweep, Hero offers everything a martial arts fan could ask for.</div>,
 <div class="info critics-consensus"><span class="descriptor">Critics Consensus:</span> Whether Road House is simply bad or so bad it's good depends largely on the audience's fondness for Swayze -- and tolerance for violently cheesy action.</div>,
 <div class="info critics-consensus"><span class="descriptor">Critics Consensus:</span> As fast, loud, and relentless as the

In [27]:
# Inspecting the text inside these tags
[con.text for con in consensus] [:5]

['Critics Consensus: Running Scared struggles to strike a consistent balance between violent action and humor, but the chemistry between its well-matched leads keeps things entertaining.',
 'Critics Consensus: Equilibrium is a reheated mishmash of other sci-fi movies.',
 'Critics Consensus: With death-defying action sequences and epic historic sweep, Hero offers everything a martial arts fan could ask for.',
 "Critics Consensus: Whether Road House is simply bad or so bad it's good depends largely on the audience's fondness for Swayze -- and tolerance for violently cheesy action.",
 "Critics Consensus: As fast, loud, and relentless as the train at the center of the story, Unstoppable is perfect popcorn entertainment -- and director Tony Scott's best movie in years."]

### Removing common phrase: 'Critics Consensus: '

#### Way 1: Text processing

In [28]:
# The simplest (but not necessarily the best) way of achieving it is by taking the substring after the common phrase

# Defining the phrase to be removed (note the space at the end)
common_phrase = 'Critics Consensus: '

# Finding how long is the common phrase
len(common_phrase)

19

In [29]:
consensus[0].text

'Critics Consensus: Running Scared struggles to strike a consistent balance between violent action and humor, but the chemistry between its well-matched leads keeps things entertaining.'

In [30]:
# Taking only the part of the text after the common phrase
consensus[0].text[19:]

'Running Scared struggles to strike a consistent balance between violent action and humor, but the chemistry between its well-matched leads keeps things entertaining.'

In [31]:
# Define a variable to store the length
common_len = len(common_phrase)

# Cleaning the list of the common phrase
consensus_text = [con.text[common_len:] for con in consensus]
consensus_text [:5]

['Running Scared struggles to strike a consistent balance between violent action and humor, but the chemistry between its well-matched leads keeps things entertaining.',
 'Equilibrium is a reheated mishmash of other sci-fi movies.',
 'With death-defying action sequences and epic historic sweep, Hero offers everything a martial arts fan could ask for.',
 "Whether Road House is simply bad or so bad it's good depends largely on the audience's fondness for Swayze -- and tolerance for violently cheesy action.",
 "As fast, loud, and relentless as the train at the center of the story, Unstoppable is perfect popcorn entertainment -- and director Tony Scott's best movie in years."]

In [32]:
# We can add if-else logic to only truncate the string in case it starts with the common phrase
consensus_text = [con.text[common_len:] if con.text.startswith(common_phrase) else con.text for con in consensus ]
consensus_text [:5]

['Running Scared struggles to strike a consistent balance between violent action and humor, but the chemistry between its well-matched leads keeps things entertaining.',
 'Equilibrium is a reheated mishmash of other sci-fi movies.',
 'With death-defying action sequences and epic historic sweep, Hero offers everything a martial arts fan could ask for.',
 "Whether Road House is simply bad or so bad it's good depends largely on the audience's fondness for Swayze -- and tolerance for violently cheesy action.",
 "As fast, loud, and relentless as the train at the center of the story, Unstoppable is perfect popcorn entertainment -- and director Tony Scott's best movie in years."]

#### Way #2: Inspecting the HTML

#### Observation
- When inspecting the HTML we see that the common phrase ("Critics Consensus: ")
- is located inside a span element
- The string we want to obtain follows that
- In my opinion, this method is closer to the BeautifulSoup approach

In [33]:
consensus[0]

<div class="info critics-consensus"><span class="descriptor">Critics Consensus:</span> Running Scared struggles to strike a consistent balance between violent action and humor, but the chemistry between its well-matched leads keeps things entertaining.</div>

In [34]:
# We can use .contents to obtain a list of all children of the tag
consensus[0].contents

[<span class="descriptor">Critics Consensus:</span>,
 ' Running Scared struggles to strike a consistent balance between violent action and humor, but the chemistry between its well-matched leads keeps things entertaining.']

In [35]:
# The second element of that list is the text we want
consensus[0].contents[1]

' Running Scared struggles to strike a consistent balance between violent action and humor, but the chemistry between its well-matched leads keeps things entertaining.'

In [36]:
# We can remove the extra whitespace (space at the beginning) with the .strip() method
consensus[0].contents[1].strip()

'Running Scared struggles to strike a consistent balance between violent action and humor, but the chemistry between its well-matched leads keeps things entertaining.'

In [37]:
# Processing all texts
consensus_text = [con.contents[1].strip() for con in consensus]
consensus_text [:5]

['Running Scared struggles to strike a consistent balance between violent action and humor, but the chemistry between its well-matched leads keeps things entertaining.',
 'Equilibrium is a reheated mishmash of other sci-fi movies.',
 'With death-defying action sequences and epic historic sweep, Hero offers everything a martial arts fan could ask for.',
 "Whether Road House is simply bad or so bad it's good depends largely on the audience's fondness for Swayze -- and tolerance for violently cheesy action.",
 "As fast, loud, and relentless as the train at the center of the story, Unstoppable is perfect popcorn entertainment -- and director Tony Scott's best movie in years."]

## 5. Directors

In [38]:
# Extracting all director divs
directors = [div.find("div", class_ = 'director') for div in divs]
directors [:5]

[<div class="info director">
 <span class="descriptor">Directed By:</span> <a class="" href="/celebrity/peter_hyams/">Peter Hyams</a></div>,
 <div class="info director">
 <span class="descriptor">Directed By:</span> <a class="" href="/celebrity/kurt_wimmer/">Kurt Wimmer</a></div>,
 <div class="info director">
 <span class="descriptor">Directed By:</span> <a class="" href="/celebrity/zhang_yimou/">Zhang Yimou</a></div>,
 <div class="info director">
 <span class="descriptor">Directed By:</span> <a class="" href="/celebrity/rowdy_herrington/">Rowdy Herrington</a></div>,
 <div class="info director">
 <span class="descriptor">Directed By:</span> <a class="" href="/celebrity/tony_scott/">Tony Scott</a></div>]

In [39]:
# Inspecting a div
directors[0]

<div class="info director">
<span class="descriptor">Directed By:</span> <a class="" href="/celebrity/peter_hyams/">Peter Hyams</a></div>

### Filtering Directors
- Notice that one link is None - the director of Iron Man is missing!
- This means we can't simply use .string,
- Because None has no string attribute
- Running the line below will raise an error if uncommented
- #[director.find("a").string for director in directors]

In [40]:
# The director's name can be found as the string of a link

# Obtaining all director links
[director.find("a") for director in directors] [40:45]

[<a class="" href="/celebrity/1040928-john_carpenter/">John Carpenter</a>,
 <a class="" href="/celebrity/renny_harlin/">Renny Harlin</a>,
 None,
 <a class="" href="/celebrity/pegg_and_edgar_wright_shaun_of_the_dead/">Edgar Wright</a>,
 <a class="" href="/celebrity/walter_hill/">Walter Hill</a>]

### Cleaning Missing Values

In [41]:
# We can use if-else to deal with the None value

final_directors = [None if director.find("a") is None else director.find("a").string for director in directors]
final_directors [:5]

['Peter Hyams', 'Kurt Wimmer', 'Zhang Yimou', 'Rowdy Herrington', 'Tony Scott']

## 6. Cast info
- Each cast member's name is the string of a link
- There are multiple cast members for a movie

In [42]:
cast_info = [div.find("div", class_ = 'cast') for div in divs]
cast_info [:5]

[<div class="info cast">
 <span class="descriptor">Starring:</span> <a class="" href="/celebrity/gregory_hines/">Gregory Hines</a>, <a class="" href="/celebrity/billy_crystal/">Billy Crystal</a>, <a class="" href="/celebrity/jimmy_smits/">Jimmy Smits</a>, <a class="" href="/celebrity/steven_bauer/">Steven Bauer</a></div>,
 <div class="info cast">
 <span class="descriptor">Starring:</span> <a class="" href="/celebrity/christian_bale/">Christian Bale</a>, <a class="" href="/celebrity/emily_watson/">Emily Watson</a>, <a class="" href="/celebrity/taye_diggs/">Taye Diggs</a>, <a class="" href="/celebrity/angus_macfadyen/">Angus Macfadyen</a></div>,
 <div class="info cast">
 <span class="descriptor">Starring:</span> <a class="" href="/celebrity/jet_li/">Jet Li</a>, <a class="" href="/celebrity/leung_chiu_wai/">Tony Leung Chiu Wai</a>, <a class="" href="/celebrity/maggie_cheung_manyuk/">Maggie Cheung</a>, <a class="" href="/celebrity/daoming_chen/">Daoming Chen</a></div>,
 <div class="info ca

In [43]:
cast_info[0]

<div class="info cast">
<span class="descriptor">Starring:</span> <a class="" href="/celebrity/gregory_hines/">Gregory Hines</a>, <a class="" href="/celebrity/billy_crystal/">Billy Crystal</a>, <a class="" href="/celebrity/jimmy_smits/">Jimmy Smits</a>, <a class="" href="/celebrity/steven_bauer/">Steven Bauer</a></div>

### Filtering the names from 1 link

In [44]:
# Let's first practice with a single movie

# Obtain all the links to different cast members
cast_links = cast_info[0].find_all('a')
cast_links

[<a class="" href="/celebrity/gregory_hines/">Gregory Hines</a>,
 <a class="" href="/celebrity/billy_crystal/">Billy Crystal</a>,
 <a class="" href="/celebrity/jimmy_smits/">Jimmy Smits</a>,
 <a class="" href="/celebrity/steven_bauer/">Steven Bauer</a>]

In [45]:
cast_names = [link.string for link in cast_links]
cast_names

['Gregory Hines', 'Billy Crystal', 'Jimmy Smits', 'Steven Bauer']

In [46]:
# OPTIONALLY: We can stitch all names together as one string

# This can be done using the join method
# To use join, pick a string to use as a separator (in our case a comma, followed with a space) and
# pass the list of strings you want to merge to the join method

cast = ", ".join(cast_names)
cast

'Gregory Hines, Billy Crystal, Jimmy Smits, Steven Bauer'

### Filtering names from multiple links
- We can either use a for loop (clearer), or
- use a nested list compehension (more concise)

#### For Loop

In [47]:
# Initialize the list of all cast memners
cast = []

# Just put all previous operations inside a for loop
for c in cast_info:
    cast_links = c.find_all('a')
    cast_names = [link.string for link in cast_links]
    
    cast.append(", ".join(cast_names)) # Joining is optional

cast [:5]

['Gregory Hines, Billy Crystal, Jimmy Smits, Steven Bauer',
 'Christian Bale, Emily Watson, Taye Diggs, Angus Macfadyen',
 'Jet Li, Tony Leung Chiu Wai, Maggie Cheung, Daoming Chen',
 'Patrick Swayze, Kelly Lynch, Sam Elliott, Ben Gazzara',
 'Denzel Washington, Chris Pine, Rosario Dawson, Ethan Suplee']

#### Nested list comprehension

In [48]:
# As you can see this can be done in just one line using nested list comprehension
# However, the code is harded to understand

cast = [", ".join([link.string for link in c.find_all("a")]) for c in cast_info]
cast [:5]

['Gregory Hines, Billy Crystal, Jimmy Smits, Steven Bauer',
 'Christian Bale, Emily Watson, Taye Diggs, Angus Macfadyen',
 'Jet Li, Tony Leung Chiu Wai, Maggie Cheung, Daoming Chen',
 'Patrick Swayze, Kelly Lynch, Sam Elliott, Ben Gazzara',
 'Denzel Washington, Chris Pine, Rosario Dawson, Ethan Suplee']

## 7. Adjusted Score

### Filtering Adjusted Score

In [49]:
# The adjusted scores can be found in a div with class 'info countdown-adjusted-score'
adj_scores = [div.find("div", {"class": "info countdown-adjusted-score"}) for div in divs]
adj_scores [:5]

[<div class="info countdown-adjusted-score"><span class="descriptor">Adjusted Score: </span>61.167% <span class="glyphicon glyphicon-question-sign" data-html="true" data-original-title="The Adjusted Score comes from a weighted formula (Bayesian) that we use that accounts for variation in the number of reviews per movie." data-placement="top" data-toggle="tooltip" rel="tooltip" title=""></span></div>,
 <div class="info countdown-adjusted-score"><span class="descriptor">Adjusted Score: </span>41.989% <span class="glyphicon glyphicon-question-sign" data-html="true" data-original-title="The Adjusted Score comes from a weighted formula (Bayesian) that we use that accounts for variation in the number of reviews per movie." data-placement="top" data-toggle="tooltip" rel="tooltip" title=""></span></div>,
 <div class="info countdown-adjusted-score"><span class="descriptor">Adjusted Score: </span>100.825% <span class="glyphicon glyphicon-question-sign" data-html="true" data-original-title="The A

In [50]:
# Inspecting an element
adj_scores[0]

<div class="info countdown-adjusted-score"><span class="descriptor">Adjusted Score: </span>61.167% <span class="glyphicon glyphicon-question-sign" data-html="true" data-original-title="The Adjusted Score comes from a weighted formula (Bayesian) that we use that accounts for variation in the number of reviews per movie." data-placement="top" data-toggle="tooltip" rel="tooltip" title=""></span></div>

In [51]:
# By inspection we see that the string we are looking for is the second child of the 'div' tag
adj_scores[0].contents[1]  # Note the extra whitespace at the end

'61.167% '

### Removing '%' and extra space ' '

In [52]:
# Extracting the string (without '%' sign and extra space)
adj_scores_clean = [score.contents[1].strip('% ') for score in adj_scores]
adj_scores_clean [:5]

['61.167', '41.989', '100.825', '41.991', '91.512']

In [53]:
# Converting the strings to numbers
final_adj = [float(score) for score in adj_scores_clean] # Note that this time the scores are float, not int!
final_adj [:5]

[61.167, 41.989, 100.825, 41.991, 91.512]

## 8. Synopsis

### Filtering Synopsis

In [54]:
# The synopsis is located inside a 'div' tag with the class 'info synopsis'
synopsis = [div.find('div', class_='synopsis') for div in divs]
synopsis [:5]

[<div class="info synopsis"><span class="descriptor">Synopsis:</span> Distinguished by a sharp, witty dialogue between its two cop protagonists, Ray and Danny (Gregory Hines and Billy Crystal), this...<a class="" data-pageheader="" href="https://www.rottentomatoes.com/m/1018009-running_scared/" target="_top"> [More]</a></div>,
 <div class="info synopsis"><span class="descriptor">Synopsis:</span> In the nation of Libria, there is always peace among men. The rules of the Librian system are simple. If...<a class="" data-pageheader="" href="https://www.rottentomatoes.com/m/equilibrium/" target="_top"> [More]</a></div>,
 <div class="info synopsis"><span class="descriptor">Synopsis:</span> Hero is two-time Academy Award nominee Zhang Yimou's directorial attempt at exploring the concept of a Chinese hero. During the...<a class="" data-pageheader="" href="https://www.rottentomatoes.com/m/hero/" target="_top"> [More]</a></div>,
 <div class="info synopsis"><span class="descriptor">Synopsis:</spa

In [55]:
# Inspecting the element
synopsis[0]

<div class="info synopsis"><span class="descriptor">Synopsis:</span> Distinguished by a sharp, witty dialogue between its two cop protagonists, Ray and Danny (Gregory Hines and Billy Crystal), this...<a class="" data-pageheader="" href="https://www.rottentomatoes.com/m/1018009-running_scared/" target="_top"> [More]</a></div>

In [56]:
# The text is the second child
synopsis[0].contents[1]

' Distinguished by a sharp, witty dialogue between its two cop protagonists, Ray and Danny (Gregory Hines and Billy Crystal), this...'

In [57]:
# Extracting the text
synopsis_text = [syn.contents[1] for syn in synopsis]
synopsis_text [:5]

[' Distinguished by a sharp, witty dialogue between its two cop protagonists, Ray and Danny (Gregory Hines and Billy Crystal), this...',
 ' In the nation of Libria, there is always peace among men. The rules of the Librian system are simple. If...',
 " Hero is two-time Academy Award nominee Zhang Yimou's directorial attempt at exploring the concept of a Chinese hero. During the...",
 ' Dalton (Swayze) is a true gentleman with a degree in philosophy from NYU. He also has a flip side -...',
 ' In this action thriller from director Tony Scott, rookie train operator Will (Chris Pine) and grizzled veteran engineer Frank (Denzel...']

# Representing the data in structured form
- We will take advantage of pandas and its dataframe for data storage

## Creating a Data Frame

In [58]:
# A dataframe is a tabular data type, frequently used in data science

movies_info = pd.DataFrame()
movies_info  # The dataframe is still empty, we need to fill it with the info we gathered

""


## Populating the dataframe

In [59]:
movies_info["Movie Title"] = movie_names
movies_info["Year"] = years
movies_info["Score"] = scores
movies_info["Adjusted Score"] = final_adj
movies_info["Director"] = final_directors
movies_info["Synopsis"] = synopsis_text
movies_info["Cast"] = cast
movies_info["Consensus"] = consensus_text
movies_info.head()

,Movie Title,Year,Score,Adjusted Score,Director,Synopsis,Cast,Consensus
0,Running Scared,1986,60,61.167,Peter Hyams,"Distinguished by a sharp, witty dialogue betw...","Gregory Hines, Billy Crystal, Jimmy Smits, Ste...",Running Scared struggles to strike a consisten...
1,Equilibrium,2002,40,41.989,Kurt Wimmer,"In the nation of Libria, there is always peac...","Christian Bale, Emily Watson, Taye Diggs, Angu...",Equilibrium is a reheated mishmash of other sc...
2,Hero,2004,95,100.825,Zhang Yimou,Hero is two-time Academy Award nominee Zhang ...,"Jet Li, Tony Leung Chiu Wai, Maggie Cheung, Da...",With death-defying action sequences and epic h...
3,Road House,1989,39,41.991,Rowdy Herrington,Dalton (Swayze) is a true gentleman with a de...,"Patrick Swayze, Kelly Lynch, Sam Elliott, Ben ...",Whether Road House is simply bad or so bad it'...
4,Unstoppable,2010,86,91.512,Tony Scott,In this action thriller from director Tony Sc...,"Denzel Washington, Chris Pine, Rosario Dawson,...","As fast, loud, and relentless as the train at ..."


In [60]:
# By default pandas abbreviates any text beyond a certain length (as seen in the Cast and Consensus columns)

# We can change that by setting the maximum column width to -1,
# which means the column would be as wide as to display the whole text
pd.set_option('display.max_colwidth', -1)
movies_info.head()

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:5: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  """


,Movie Title,Year,Score,Adjusted Score,Director,Synopsis,Cast,Consensus
0,Running Scared,1986,60,61.167,Peter Hyams,"Distinguished by a sharp, witty dialogue between its two cop protagonists, Ray and Danny (Gregory Hines and Billy Crystal), this...","Gregory Hines, Billy Crystal, Jimmy Smits, Steven Bauer","Running Scared struggles to strike a consistent balance between violent action and humor, but the chemistry between its well-matched leads keeps things entertaining."
1,Equilibrium,2002,40,41.989,Kurt Wimmer,"In the nation of Libria, there is always peace among men. The rules of the Librian system are simple. If...","Christian Bale, Emily Watson, Taye Diggs, Angus Macfadyen",Equilibrium is a reheated mishmash of other sci-fi movies.
2,Hero,2004,95,100.825,Zhang Yimou,Hero is two-time Academy Award nominee Zhang Yimou's directorial attempt at exploring the concept of a Chinese hero. During the...,"Jet Li, Tony Leung Chiu Wai, Maggie Cheung, Daoming Chen","With death-defying action sequences and epic historic sweep, Hero offers everything a martial arts fan could ask for."
3,Road House,1989,39,41.991,Rowdy Herrington,Dalton (Swayze) is a true gentleman with a degree in philosophy from NYU. He also has a flip side -...,"Patrick Swayze, Kelly Lynch, Sam Elliott, Ben Gazzara",Whether Road House is simply bad or so bad it's good depends largely on the audience's fondness for Swayze -- and tolerance for violently cheesy action.
4,Unstoppable,2010,86,91.512,Tony Scott,"In this action thriller from director Tony Scott, rookie train operator Will (Chris Pine) and grizzled veteran engineer Frank (Denzel...","Denzel Washington, Chris Pine, Rosario Dawson, Ethan Suplee","As fast, loud, and relentless as the train at the center of the story, Unstoppable is perfect popcorn entertainment -- and director Tony Scott's best movie in years."


## Exporting the data to CSV (comma-separated values) and excel files
- Index is set to False so that the index (0,1,2...) of each movie is not saved to the file (the index is purely internal)
- The header is set to True, so that the names of the columns are saved

In [61]:
# Write data to CSV file
movies_info.to_csv("movies_info.csv", index = False, header = True)

# Write data to excel file
#movies_info.to_excel("movies_info.xlsx", index = False, header = True)